In [1]:
import json
import pandas as pd
import numpy as np
import os
import time
import requests

logic: 
1. at the last block of current epoch, get the address of all the eligible validators
2. at the first block of next epoch, group all the eligible validators into elected and non-elected
3. sort elected and non-elected groups based on their stakes
4. compare the lowest stake in elected group and highest stake in non-elected group. 
expectation: lowest stake in elected group > highest stake in non-elected group

Question: if two validators have the same stake and both active, their stake rank is No.200. Who will be elected? based on the order who joined the network first?

In [2]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [3]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [4]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [5]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [0]
    return get_information(method, params)['result']

In [6]:
def getEligibleValidator():
    eligible = []
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['epos-status'] == 'currently elected and signing enough blocks to be eligible for election next epoch' or\
        i['epos-status'] == 'eligible to be elected next epoch':
            address = i['validator']['address']
            eligible.append(address)
    return eligible

In [7]:
def getValidatorInfo(validator):
    method = "hmy_getValidatorInformation"
    params = [validator]
    return get_information(method, params)['result']

In [8]:
def getStake(eligible):
    elected = dict()
    non_elected = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        address = i['validator']['address']
        if address in eligible:
            stake = i['total-delegation']
            if i['currently-in-committee'] == True:
                elected[address] = stake
            else:
                non_elected[address] = stake
    return elected, non_elected

In [9]:
def E4_test():
    print("Test-E4: Low staker will never get elected over high staker")
    
    num = int(input("enter the number of epoches you want to test: "))
    # reach the last block in current epoch to get the eligible validator address
    
    iterations = 0
    while iterations < num:
        print(str(iterations+1) + "iteration will begin ...")
        # reach the last block in current epoch to get the eligible validator address
        block = getBlockNumber()
        last_block = getLastBlockOfCurrentEpoch()
        print("current and last block numbers", block, last_block)
        while block < last_block:
            block = getBlockNumber()
        eligible = getEligibleValidator()

        # reach the first block in next epoch and check the status of all eligible validators
        new_epoch_block = block + 1
        while block < new_epoch_block:
            block = getBlockNumber()
        print("new epoch first block reached", new_epoch_block, "will begin testing")

        elected, non_elected = getStake(eligible)
        sorted_elected = sorted(elected.items(), key = lambda kv: kv[1])
        sorted_non_elected = sorted(non_elected.items(), key = lambda kv: kv[1], reverse = True)

        # get the lowest elected validator and highest non-elected validator
        if not sorted_elected:
            lowest_elected = 0
        else:
            lowest_elected = sorted_elected[0][1]
        if not sorted_non_elected:
            highest_unelected = 0
        else:
            highest_unelected = sorted_non_elected[0][1]

        if lowest_elected < highest_unelected:
            print("Test-E4: Fail")
            print("lowest stake in elected eligible validators: ", sorted_elected[0])
            print("highest stake in unelected eligible validators: ", sorted_non_elected[0])
        print("Test-E4: Success")
        iterations += 1
        

In [10]:
E4_test()

Test-E4: Low staker will never get elected over high staker
enter the number of epoches you want to test: 3
1iteration will begin ...
current and last block numbers 5031 5053
new epoch first block reached 5054 will begin testing
Test-E4: Success
2iteration will begin ...
current and last block numbers 5054 5091
new epoch first block reached 5092 will begin testing
Test-E4: Success
3iteration will begin ...
current and last block numbers 5092 5129
new epoch first block reached 5130 will begin testing
Test-E4: Success
